In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import os, sys, getpass
user = getpass.getuser()
sys.path.append(f'/home/{user}/codes/hpe_library/')
from lib_import import *
from my_utils import *
os.chdir('/home/hrai/codes/MotionBERT')

### Load data_type

In [4]:
data_type_list = ['source_list', 'cam_param', 'cam_param_adaptive_focal',
                  'world_3d', 'cam_3d', 'img_2d', 
                  'img_3d', 'scale_factor', 'img_25d', 
                  'cam_3d_canonical_same_z', 'img_2d_canonical_same_z', 
                  'cam_3d_canonical_same_dist', 'img_2d_canonical_same_dist',
                  'cam_3d_canonical_fixed_dist_5', 'img_2d_canonical_fixed_dist_5', 'img_2d_canonical_fixed_dist_5_adaptive_focal'
                  ]
overwrite_list = []
dataset_name = 'h36m'
data = load_data_dict(dataset_name, data_type_list, overwrite_list, verbose=True)

[overwrite: False] ==> Loading H36M source_list...
[overwrite: False] ==> Loading H36M cam_param...
[overwrite: False] ==> Loading H36M cam_param_adaptive_focal...
[overwrite: False] ==> Loading H36M world_3d...
[overwrite: False] ==> Loading H36M cam_3d...
[overwrite: False] ==> Loading H36M img_2d...
[overwrite: False] ==> Loading H36M img_3d...
[overwrite: False] ==> Loading H36M scale_factor...
[overwrite: False] ==> Loading H36M img_25d...
[overwrite: False] ==> Loading H36M cam_3d_canonical_same_z...
[overwrite: False] ==> Loading H36M img_2d_canonical_same_z...
[overwrite: False] ==> Loading H36M cam_3d_canonical_same_dist...
[overwrite: False] ==> Loading H36M img_2d_canonical_same_dist...
[overwrite: False] ==> Loading H36M cam_3d_canonical_fixed_dist_5...
[overwrite: False] ==> Loading H36M img_2d_canonical_fixed_dist_5...
[overwrite: False] ==> Loading H36M img_2d_canonical_fixed_dist_5_adaptive_focal...


In [6]:
data.keys()

dict_keys(['source_list', 'cam_param', 'cam_param_adaptive_focal', 'world_3d', 'cam_3d', 'img_2d', 'img_3d', 'scale_factor', 'img_25d', 'cam_3d_canonical_same_z', 'img_2d_canonical_same_z', 'cam_3d_canonical_same_dist', 'img_2d_canonical_same_dist', 'cam_3d_canonical_fixed_dist_5', 'img_2d_canonical_fixed_dist_5', 'img_2d_canonical_fixed_dist_5_adaptive_focal'])

In [10]:
# check constraints
for source in data['source_list']:
    subject, cam_id, action = split_source_name(source, dataset_name)
    world_3d = data['world_3d'][subject][action]
    cam_3d = data['cam_3d'][subject][action][cam_id]
    img_2d = data['img_2d'][subject][action][cam_id]
    img_3d = data['img_3d'][subject][action][cam_id]
    cam_3d_canonical_same_z = data['cam_3d_canonical_same_z'][subject][action][cam_id]
    cam_3d_canonical_same_dist = data['cam_3d_canonical_same_dist'][subject][action][cam_id]
    cam_3d_canonical_fixed_dist_5 = data['cam_3d_canonical_fixed_dist_5'][subject][action][cam_id]
    assert world_3d.shape[1:] == (17, 3), f"{source} {data['world_3d'][subject][action].shape}"
    assert world_3d.mean() < 5, f'{source} {world_3d.mean()}'
    assert cam_3d.shape[1:] == (17, 3), f'{source} {cam_3d.shape}'
    assert cam_3d.mean() < 5, f'{source} {cam_3d.mean()}'
    assert img_2d.mean() > 1, f'{source} {img_2d.mean()}'
    assert np.allclose(img_2d, img_3d[...,:2]), f'image 2d and 3d are not equal'
    assert np.allclose(cam_3d[...,2], cam_3d_canonical_same_z[...,2]), 'Z values should be same'
    assert np.allclose(np.linalg.norm(cam_3d[:, 0], axis=1), cam_3d_canonical_same_dist[:, 0, 2]), 'Distances should be same'
    assert np.allclose(np.array([5.0]*len(cam_3d)), cam_3d_canonical_fixed_dist_5[:, 0, 2]), 'Distances should be same'